# Boston Traffic Stop Analysis
By. Andrew Simmons & Ethan Smith

In [ ]:
from operator import itemgetter

import graphviz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
"""Constants"""

SEED = 42

In [ ]:
"""Load data and filter"""

dataset = pd.read_csv('data/boston-police-department-fio.csv')

dataset.replace('NO DATA ENTERED', np.NaN, inplace=True)
dataset.replace('UNKNOWN', np.NaN, inplace=True)

dataset['VEH_YEAR_NUM'] = dataset['VEH_YEAR_NUM'].replace(0, np.NaN)
dataset['AGE_AT_FIO_CORRECTED'] = dataset['AGE_AT_FIO_CORRECTED'].replace(0, np.NaN)
dataset['AGE_AT_FIO_CORRECTED'] = dataset['AGE_AT_FIO_CORRECTED'].replace(-1, np.NaN)

In [ ]:
dataset.columns

## Data Visualization

In [ ]:
"""Sex Distibution"""

males = len(dataset[dataset['SEX'] == 'MALE'].index)
females = len(dataset[dataset['SEX'] == 'FEMALE'].index)

sexes = ['Male', 'Female']
counts = np.array([males, females])

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))

sex_distribution = ax.pie(counts,
                           explode=[0, 0.15],
                           shadow=True,
                           labels=sexes,
                           autopct="%1.1f%%")

ax.set_title('Sex Distribution')

In [ ]:
"""Prior Offenses"""

has_prior_offenses = len(dataset[dataset['PRIORS'] == 'YES'].index)
no_prior_offenses = len(dataset[dataset['PRIORS'] == 'NO'].index)

labels = ['Yes', 'No']
counts = [has_prior_offenses, no_prior_offenses]

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))

sex_distribution = ax.pie(counts,
                           explode=[0, 0.15],
                           shadow=True,
                           labels=labels,
                           autopct="%1.1f%%")

ax.set_title('Prior Offenses Distribution')

In [ ]:
"""Terrorism Distribution"""

terrorism = len(dataset[dataset['TERRORISM'] == 'YES'].index)
no_terrorism = len(dataset[dataset['TERRORISM'] == 'NO'].index)

labels = ['Terrorism', 'Non-Terrorism']
counts = [terrorism, no_terrorism]

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))

sex_distribution = ax.pie(counts,
                           shadow=True,
                           labels=labels,
                           autopct="%1.1f%%")

ax.set_title('Terrorism Distribution')

In [ ]:
"""Vehicle Make Distribution"""

without_null_vehicle_makes = dataset[~pd.isnull(dataset['VEH_MAKE'])]
vehicle_makes = list(without_null_vehicle_makes['VEH_MAKE'].unique())

counts = []
for make in vehicle_makes:
    counts.append(len(without_null_vehicle_makes[without_null_vehicle_makes['VEH_MAKE'] == make].index))
    
make_counts = sorted(zip(vehicle_makes, counts), key=itemgetter(1), reverse=True)
makes, counts = zip(*make_counts)

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))

sex_distribution = ax.bar(makes, counts)

ax.set_title('Vehicle Make Distribution')

for tick in ax.get_xticklabels():
    tick.set_rotation(90)

In [ ]:
"""Race Distribution"""

without_null_race = dataset[~pd.isnull(dataset['RACE_DESC'])]
races = list(without_null_race['RACE_DESC'].unique())

counts = []
for race in races:
    counts.append(len(without_null_race[without_null_race['RACE_DESC'] == race].index))

race_counts = sorted(zip(races, counts), key=itemgetter(1), reverse=True)
races, counts = zip(*race_counts)

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))

sex_distribution = ax.bar(races, counts)

ax.set_title('Race Distribution')

for tick in ax.get_xticklabels():
    tick.set_rotation(90)

In [ ]:
"""Race Distribution Without Priors"""

without_null_race = dataset[~pd.isnull(dataset['RACE_DESC'])]
without_priors = without_null_race[without_null_race['PRIORS'] == 'NO']
races = list(without_priors['RACE_DESC'].unique())

counts = []
for race in races:
    counts.append(len(without_priors[without_priors['RACE_DESC'] == race].index))

race_counts = sorted(zip(races, counts), key=itemgetter(1), reverse=True)
races, counts = zip(*race_counts)

fig, ax = plt.subplots()
fig.set_size_inches((10, 10))

sex_distribution = ax.bar(races, counts)

ax.set_title('Race Distribution Without Priors')

for tick in ax.get_xticklabels():
    tick.set_rotation(90)

In [ ]:
"""Decision Tree"""

LABEL_FEATURE = 'PRIORS'

# Select columns to include in model
important_features = dataset[
    [
        'SEX',
        'PRIORS',
        'COMPLEXION',
        'TERRORISM',
        'BASIS',
        'STOP_REASONS',
        'FIOFS_REASONS',
        'VEH_MAKE',
        'VEH_YEAR_NUM',
        'VEH_COLOR',
        'VEH_OCCUPANT',
        'VEH_STATE',
        'RACE_DESC',
        'AGE_AT_FIO_CORRECTED',
        'CITY',
    ]
].copy()
# TODO: Conside the following features:
#     FIOFS_TYPE: Figure out what these mean
#     SEARCH: Figure out what these mean
#     OUTCOME: Figure out what these mean

# Drop rows where LABEL_FEATURE is NaN
important_features.dropna(subset=[LABEL_FEATURE], inplace=True)

# Fill NaN values with either mode or mean
for column_name in important_features.columns:
    column = important_features[column_name]
    column_dtype = column.dtype
    
    if column_dtype == np.dtype(np.object):
        column.fillna(column.mode()[0], inplace=True)
    else:
        column.fillna(column.mean(), inplace=True)

# Create LabelEncoder for each catagorical column
encoders = {}
for column_name in important_features.columns:
    column = important_features[column_name]
    column_dtype = column.dtype
    
    # Skip if numeric column
    if column_dtype != np.dtype(np.object):
        continue
    
    encoder = LabelEncoder()
    encoder.fit(column.unique())
    
    encoders[column_name] = encoder
    
    transformed_column = encoder.transform(column)
    important_features[column_name] = transformed_column

# TODO: Remove Debug
important_features.to_csv('data/cleaned_decision_tree_features.csv', index=False)

classifier = DecisionTreeClassifier(random_state=SEED)

X = important_features.drop(LABEL_FEATURE, axis=1)
y = important_features[LABEL_FEATURE]

print(np.mean(cross_val_score(classifier, X, y, cv=5)))

In [ ]:
yes = len(dataset[dataset['PRIORS'] == 'YES'])
no = len(dataset[dataset['PRIORS'] == 'NO'])
total = yes + no

print(yes / total)